In [1]:
import numpy as np
import pandas as pd
import requests
import re
import os

In [241]:
df = pd.read_csv('P3_GrantExport.csv', sep=';', index_col=0, na_values=['Nicht zuteilbar - NA'])

In [242]:
df.index.is_unique

True

In [244]:
df.University.describe()

count                       48393
unique                         76
top       Universität Zürich - ZH
freq                         6774
Name: University, dtype: object

In [271]:
def trim_university_name(full_name):
    match = re.match('^(.+?)(?: - .+)?$', full_name)
    if match is None:
        return None
    else:
        return match.group(1)

In [374]:
uni_names = df.University.drop_duplicates().dropna().apply(lambda s: trim_university_name(s))

In [398]:
search_uni('Universität Basel')

{'error_message': 'You have exceeded your daily request quota for this API.', 'html_attributions': [], 'results': [], 'status': 'OVER_QUERY_LIMIT'}


nan

In [389]:
t = uni_names.head(3)

In [397]:
def create_columns(uni_name):
    column_names = ('Canton', 'Latitude', 'Longitude')
    data = search_uni(uni_name)
    
    return pd.Series(dict(zip(column_names, data)))
    
t.apply(create_columns).set_index(t)

{'html_attributions': [], 'results': [{'photos': [{'width': 3840, 'photo_reference': 'CoQBcwAAALfNFcUVk2cppfQFFQ3sp1rbDryLJMz_XALB5-jgBl_qMrFLJyI_uwLcabMcJPfIW66g390jaL4Gkv5-_nf4jDRhEQRCjMiUthghJsc5oyiKaBy32IMqv91EQ6xQy9pQC_SyLVG6vuX-DYPFzBSauMs5f-pe1l0LS4Fkoe_HKjeHEhBz-ijScLbBjQOlDbLsfkDPGhTSthQyxCq7J5OHT1fA8IZl4G61QQ', 'height': 2160, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107958259135711194844/photos">Balázs Fodor</a>']}], 'place_id': 'ChIJ3yKsR1FkjEcRIpuYmmtbIq0', 'reference': 'CmRSAAAA-En8An3mKfVBM1V-WBzsteHdIwT3a9pzleig7yyqUZW1GJ3gzncUw6lXWV700fPzyRa35v_xEXUowBXBdRdqIFmA3bkzUePtHk3eOiNAol_BcLADhgSNONCl14U0uudHEhDFIwEX8XtOSN77Hra9N79uGhRTF0ITtP8iptO_sEwvstJiP0EsOg', 'geometry': {'viewport': {'southwest': {'lat': 46.19747575000001, 'lng': 6.142004249999998}, 'northeast': {'lat': 46.20045794999999, 'lng': 6.146152850000001}}, 'location': {'lat': 46.199444, 'lng': 6.1451157}}, 'types': ['university', 'point_of_interest', 'establishment'], 'icon': 'https:

TypeError: zip argument #2 must support iteration

In [215]:
def get_api_key():
    with open('google_api_key.json') as f:
        json_data = json.load(f)
        return json_data['api_key']

In [362]:
def search_place(place_name):
    BASE_URL = 'https://maps.googleapis.com/maps/api/place/textsearch/json'
    
    api_key = get_api_key()
    parameters = {'key': api_key, 'query': place_name}
    
    r = requests.get(BASE_URL, params=parameters)
    response = r.json()
    print(response)

    if response['status'] != 'OK':
        return None
    else:
        result = response['results'][0]
        location = result['geometry']['location']
        place_id = result['place_id']
        
        return (place_id, location['lat'], location['lng'])

In [217]:
def search_canton(place_id):
    BASE_URL = 'https://maps.googleapis.com/maps/api/geocode/json'
    
    api_key = get_api_key()
    parameters = {'key': api_key, 'place_id': place_id}
    
    r = requests.get('https://maps.googleapis.com/maps/api/geocode/json', params=parameters)
    response = r.json()
    
    if response['status'] != 'OK':
        return None
    else:
        result = response['results'][0]
        
        for component in result['address_components']:
            if 'administrative_area_level_1' in component['types']:
                return component['short_name']
            
        return None

In [218]:
def search_uni(university_name):
    place = search_place(university_name)
    if place is None:
        return np.nan
    else:
        place_id, lat, lng = place
        canton = search_canton(place_id)
        return (canton, lat, lng)

In [219]:
uni_infos = {university: search_uni(university) for university in df.University.dropna().unique()}

In [220]:
uni_infos

{'AO Research Institute - AORI': nan,
 'Allergie- und Asthmaforschung - SIAF': nan,
 'Berner Fachhochschule - BFH': nan,
 'Biotechnologie Institut Thurgau - BITG': nan,
 "Centre de rech. sur l'environnement alpin - CREALP": nan,
 'EPF Lausanne - EPFL': nan,
 'ETH Zürich - ETHZ': nan,
 'Eidg. Anstalt für Wasserversorgung - EAWAG': nan,
 'Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL': nan,
 'Eidg. Hochschulinstitut für Berufsbildung - EHB': nan,
 'Eidg. Material und Prüfungsanstalt - EMPA': nan,
 'Ente Ospedaliero Cantonale - EOC': nan,
 'Fachhochschule Kalaidos - FHKD': nan,
 'Fachhochschule Nordwestschweiz (ohne PH) - FHNW': nan,
 'Fachhochschule Ostschweiz - FHO': nan,
 'Facoltà di Teologia di Lugano - FTL': nan,
 'Fernfachhochschule Schweiz (Mitglied SUPSI) - FFHS': nan,
 'Firmen/Privatwirtschaft - FP': nan,
 'Forschungsanstalten Agroscope - AGS': nan,
 'Forschungsinstitut für Opthalmologie - IRO': nan,
 'Forschungsinstitut für biologischen Landbau - FIBL': nan,
 'Forschungskom

In [209]:
unis.count()

canton       60
latitude     60
longitude    60
dtype: int64

In [212]:
unis = pd.DataFrame(uni_infos).T
unis.columns = ['canton', 'latitude', 'longitude']
unis.index.name = 'university'
unis

,canton,latitude,longitude
university,,,
AO Research Institute - AORI,NaN,NaN,NaN
Allergie- und Asthmaforschung - SIAF,NaN,NaN,NaN
Berner Fachhochschule - BFH,NaN,NaN,NaN
Biotechnologie Institut Thurgau - BITG,NaN,NaN,NaN
Centre de rech. sur l'environnement alpin - CREALP,NaN,NaN,NaN
EPF Lausanne - EPFL,NaN,NaN,NaN
ETH Zürich - ETHZ,NaN,NaN,NaN
Eidg. Anstalt für Wasserversorgung - EAWAG,NaN,NaN,NaN
"Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL",NaN,NaN,NaN


In [211]:
unis.to_csv('universities.csv')

In [178]:
unis[unis.isnull()]

,canton,latitude,longitude
university,,,
AO Research Institute - AORI,NaN,NaN,NaN
Allergie- und Asthmaforschung - SIAF,NaN,NaN,NaN
Berner Fachhochschule - BFH,NaN,NaN,NaN
Biotechnologie Institut Thurgau - BITG,NaN,NaN,NaN
Centre de rech. sur l'environnement alpin - CREALP,NaN,NaN,NaN
EPF Lausanne - EPFL,NaN,NaN,NaN
ETH Zürich - ETHZ,NaN,NaN,NaN
Eidg. Anstalt für Wasserversorgung - EAWAG,NaN,NaN,NaN
"Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL",NaN,NaN,NaN
